# Multiple Linear Regression

- Same as linear regression but with more features involved: $$ y = b_{0} + b_{1}x_{1}+b_{2}x_{2}+...+b_{n}x_{n}$$

- Need to figure out how each independent variable ($x_{n}$) affect the dependent variable ($y$)

There are assumptions associated with Linear Regression:
- Linearity
- Homoscedasticity
- Multivariate normality
- Independence of errors
- Lack of multicollinearity

When making a real linear regression model, need to make sure these assumptions are true with respect to the data. Won't be doing that here.

Here we will try to predict if there is a correlation between profit and the amount of spend in different areas of the business and also the location of the business (NY or CA, categorical variable). Need to incluse a dummy variable (i.e NY = 1, CA = 0).

Mind the dummy varibale trap. 

## Five methods of building models:
1. All-in
2. Backward Elimination
3. Forward Selection
4. Bidirectional Elimination
5. Score Comparison

Note: 3,4 and 5 are known collectively as stepwise regression. Although most people are talking about backward elimination.

### Backward Elimination

- Step 1: select a significance level to stay in model (SL = 0.05)
- Step 2: Fit the full model with all possible predictors
- Step 3: Consider predictor with highest p-value. If p > SL, go to step 4, otherwise finish
- Step 4: Remove predictor
- Step 5: Fit model without this variable - GO BACK TO STEP 3

### Forward Selection
- Step 1: select a significance level to enter the model (SL = 0.05)
- Step 2: Fit all simple regression models. Select one with lowest p value
- Step 3: Keep this variable and fit all possible models with one extra predictor
- Step 4: Consider predictor with lowest p value. If P < SL, go to step 3, otherwise finish

### Bidirectional Elimination
- Step 1: select a significance level to enter and to stay in model (SLenter = 0.05, SLstay = 0.05)
- Step 2: Perform the next step of Forward Selection (new variables must have: p < SLenter)
- Step 3: Perform all steps of Backward Elimination (old variables must have P > SLstay to stay)
- Step 4: No new variables can enter and no old variables can exit. Finish.

We will look at BACKWARD ELIMINATION as it is the quickest.

In [1]:
import numpy as np
import matplotlib.pyplot as py
import pandas as pd

In [2]:
dataset = pd.read_csv('50_Startups.csv')
dataset.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [3]:
# split data into matrix of features (X) and independent vector (y)
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,4].values

# need to encode categorical variables (State = CA, NY or FL), index 3

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_state = LabelEncoder()
X[:,3] = labelencoder_state.fit_transform(X[:, 3])
onehotencoder = OneHotEncoder(categorical_features = [3])
X = onehotencoder.fit_transform(X).toarray()
X[0:3]

array([[0.0000000e+00, 0.0000000e+00, 1.0000000e+00, 1.6534920e+05,
        1.3689780e+05, 4.7178410e+05],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.6259770e+05,
        1.5137759e+05, 4.4389853e+05],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 1.5344151e+05,
        1.0114555e+05, 4.0793454e+05]])

Can see from the new X array that 3 new dummy variables have been included as the first 3 columns, each with a value of 1 or 0 to indicate which state the business is from.

X[0] is 0,0,1 which is NY
X[1] is 1,0,0 which is CA
X[2] is 0,1,0 which is FL

So to avoid the dummy variable trap, need to remove one column. Should always be n-1 in your dataset. 

In [4]:
#Remove first column - although the python library does this for you
X =X[:,1:]

#important to note X is a matrix of features (i.e. multidimensional) and y is a vector (i.e 1D)
X.shape, y.shape

((50, 5), (50,))

In [5]:
# split the dataset into train and test
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Fit multiple linear regression model to train data
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [6]:
#test model on test data
y_pred = regressor.predict(X=X_test)
for val1, val2 in zip(y_pred, y_test):
    print('prediction = {}, real = {}'.format(val1, val2))

prediction = 103015.20159796353, real = 103282.38
prediction = 132582.2776081537, real = 144259.4
prediction = 132447.73845174702, real = 146121.95
prediction = 71976.09851258088, real = 77798.83
prediction = 178537.48221055616, real = 191050.39
prediction = 116161.24230165544, real = 105008.31
prediction = 67851.69209675616, real = 81229.06
prediction = 98791.73374687255, real = 97483.56
prediction = 113969.4353301297, real = 110352.25
prediction = 167921.06569550867, real = 166187.94


Just from inspection, can see some good predictions and some average ones. Need to use the metric library to see exactly how the model performed.

In [7]:
# Building the optimal model using backwards elimination
import statsmodels.formula.api as sm

# need to add column of x0 = 1 as statsmodel doesnt take this into account where as sklearn does
X = np.append(arr=np.ones((50,1)).astype(int), values=X, axis=1)
X[0]

array([1.000000e+00, 0.000000e+00, 1.000000e+00, 1.653492e+05,
       1.368978e+05, 4.717841e+05])

In [8]:
# create new matrix of feature containing on statistically relevant features
# to begin, take all features
X_opt = X[:, [0,1,2,3,4,5]]

# need to select a significance level e.g. SL = 0.05
# make new regressor from sm 
regressor_OLS = sm.OLS(endog=y, exog=X_opt).fit()

#now need to find the predictor with the highest p-values
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     169.9
Date:                Wed, 17 Jul 2019   Prob (F-statistic):           1.34e-27
Time:                        11:38:21   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      44   BIC:                             1074.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.013e+04   6884.820      7.281      0.000    3.62e+04     6.4e+04
x1           198.7888   3371.007      0.059      0.953   -6595.030    6992.607
x2           -41.8870   3256.039     -0.013      0.990   -6604.003    6520.229
x3             0.8060      0.046     17.369      0.000       0.712       0.900
x4            -0.0270      0.052     -0.517      0.608      -0.132       0.078
x5             0.0270      0.017      1.574      0.123      -0.008       0.062
==============================================================================
Omnibus:                       14.782   Durbin-Watson:                   1.283
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.266
Skew:                          -0.948   Prob(JB):                     2.41e-05
Kurtosis:                       5.572   Cond. No.                     1.45e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.45e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Can see from the $P>|t|$ that x2 has the largest p_value of 0.990 which is much large than the SL 0.05. So need to remove x2, which is the second dummy variable for the state.

In [9]:
X_opt = X[:, [0,1,3,4,5]]
regressor_OLS = sm.OLS(endog=y, exog=X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     217.2
Date:                Wed, 17 Jul 2019   Prob (F-statistic):           8.49e-29
Time:                        11:42:50   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1061.
Df Residuals:                      45   BIC:                             1070.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.011e+04   6647.870      7.537      0.000    3.67e+04    6.35e+04
x1           220.1585   2900.536      0.076      0.940   -5621.821    6062.138
x2             0.8060      0.046     17.606      0.000       0.714       0.898
x3            -0.0270      0.052     -0.523      0.604      -0.131       0.077
x4             0.0270      0.017      1.592      0.118      -0.007       0.061
==============================================================================
Omnibus:                       14.758   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.172
Skew:                          -0.948   Prob(JB):                     2.53e-05
Kurtosis:                       5.563   Cond. No.                     1.40e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Now need to get rid of x1 as 0.940 < 0.05. x1 is first dummy variable for the state.

In [10]:
X_opt = X[:, [0,3,4,5]]
regressor_OLS = sm.OLS(endog=y, exog=X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     296.0
Date:                Wed, 17 Jul 2019   Prob (F-statistic):           4.53e-30
Time:                        11:45:32   Log-Likelihood:                -525.39
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      46   BIC:                             1066.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.012e+04   6572.353      7.626      0.000    3.69e+04    6.34e+04
x1             0.8057      0.045     17.846      0.000       0.715       0.897
x2            -0.0268      0.051     -0.526      0.602      -0.130       0.076
x3             0.0272      0.016      1.655      0.105      -0.006       0.060
==============================================================================
Omnibus:                       14.838   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.442
Skew:                          -0.949   Prob(JB):                     2.21e-05
Kurtosis:                       5.586   Cond. No.                     1.40e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Now need to get rid of x2 as 0.602 < 0.05. x2 is admin spend (or column 4 in X).

In [11]:
X_opt = X[:, [0,3,5]]
regressor_OLS = sm.OLS(endog=y, exog=X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     450.8
Date:                Wed, 17 Jul 2019   Prob (F-statistic):           2.16e-31
Time:                        11:46:49   Log-Likelihood:                -525.54
No. Observations:                  50   AIC:                             1057.
Df Residuals:                      47   BIC:                             1063.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.698e+04   2689.933     17.464      0.000    4.16e+04    5.24e+04
x1             0.7966      0.041     19.266      0.000       0.713       0.880
x2             0.0299      0.016      1.927      0.060      -0.001       0.061
==============================================================================
Omnibus:                       14.677   Durbin-Watson:                   1.257
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.161
Skew:                          -0.939   Prob(JB):                     2.54e-05
Kurtosis:                       5.575   Cond. No.                     5.32e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.32e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Now need to get rid of x2 as 0.06 < 0.05. x2 is the marketing spend (or column 5 in X).

In [13]:
X_opt = X[:, [0,3]]
regressor_OLS = sm.OLS(endog=y, exog=X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     849.8
Date:                Wed, 17 Jul 2019   Prob (F-statistic):           3.50e-32
Time:                        11:48:32   Log-Likelihood:                -527.44
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      48   BIC:                             1063.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.903e+04   2537.897     19.320      0.000    4.39e+04    5.41e+04
x1             0.8543      0.029     29.151      0.000       0.795       0.913
==============================================================================
Omnibus:                       13.727   Durbin-Watson:                   1.116
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               18.536
Skew:                          -0.911   Prob(JB):                     9.44e-05
Kurtosis:                       5.361   Cond. No.                     1.65e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.65e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

From Backward Elimination we can see that the R&D spend is the only statistically significant feature that can be used to predict the profit.

In [14]:
# nice feature would be to automate the backwards elimination
def backwardElimination(x, sl):
    numVars = len(x[0])
    for i in range(0, numVars):
        regressor_OLS = sm.OLS(y, x).fit()
        maxVar = max(regressor_OLS.pvalues).astype(float)
        if maxVar > sl:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):
                    x = np.delete(x, j, 1)
    regressor_OLS.summary()
    return x

SL = 0.05
X_opt = X[:, [0, 1, 2, 3, 4, 5]]
X_Modeled = backwardElimination(X_opt, SL)

#to use make sure kernel has been restarted!